In [ ]:
# dependencies
!pip install boto3>=1.28.59
!pip install anthropic
!pip install langchain
!pip install langchainhub
!pip install langchain_community

In [ ]:
# global imports
import json
import os
import sys
import logging
import boto3
import botocore

logger = logging.getLogger(__name__)

module_path = ".."
sys.path.append(os.path.abspath(module_path))

In [ ]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.messages import HumanMessage

chat = BedrockChat(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs={"temperature": 0.1},
)

messages = [
    HumanMessage(
        content="Translate this sentence from English to Chinese. I love programming."
    )
]
msg=chat(messages)
print(msg.content)

In [ ]:
werewolf_rule_v1 = """
1. 游戏分坏人(狼人)和好人(村民+预言家)两大阵营,他们的目标为:
- 坏人阵营只有狼人
- 好人阵营有村民和预言家
- 坏人阵营:消灭所有好人, 或者保证坏人数目大于好人数目
- 好人阵营:消灭所有坏人, 或者保证好人数目大于坏人数目

2. 游戏分白天和晚上两个阶段交替进行:
- 晚上狼人睁眼统一投票杀死一名玩家
- 晚上预言家只能查验一名玩家身份
- 晚上普通村民无法行动
- 白天分为讨论和投票两环节
- 白天在讨论环节，每个玩家必须参与讨论
- 白天在投票环节，每个玩家必须投票或者放弃

"""

werewolf_command_v1 = """
- WolfVote: 夜晚投票(狼人专属行动),参数: target=存活玩家
- ProphetCheck: 夜晚查验(预言家专属行动), 参数: target=存活玩家
- PlayerDoubt: 白天怀疑(所有玩家白天可选行动, 非投票), 参数: target=存活玩家 
- PlayerVote: 白天投票, 参数: target=存活玩家 
- Debate: 白天讨论, 参数: content=思考/理由 
- GetAllPlayersName: 玩家信息, 参数: 无 
- DeathWords: 死亡遗言, 参数: content=给予玩家线索
- Pass: 玩家弃权参数: 无 
"""

template_player_role = """你是资深的社交游戏玩家, 熟悉《狼人杀》游戏规则:
<game_rules>
{game_rule}
</game_rules>

你熟悉该游戏所有命令:
<commands>
{commands}
</commands>

<reflections>
- 按照游戏规则，第一个夜晚死亡的一定是村民或者预言家，狼人没必要第一晚上自杀
- 看完P3玩家昨天白天投票，明显感觉他在混淆是非，很有可能在给狼人分票
- 平民玩家由于信息缺失，所以狼人要尽量引导他们去集火其他人
- 第一个夜晚所有的行动都是随机的
- 作为第一个死亡的玩家，其实信息有限，我就靠第六感推理...
</reflections>

<references>
- {{"action": "Pass"}}
- {{"action": "WolfVote", "target": "小明"}}
- {{"action": "ProphetCheck", "target": "P1"}}
- {{"action": "PlayerVote", "target": "老王"}}
- {{"action": "Debate", "content": "我的推理为xx是狼，原因是..."}}
- {{"action": "DeathWords", "content": "我觉得xx有很大的嫌疑, 原因是..."}}
</references>

历史信息:
<history>
{history}
</history>

记住，你支持的玩家是 {nickname}, 身份是 {role}, 性格为 {character}, 必须帮助玩家进行这个游戏
接下来你的目的是: 通过一步一步思考决策引导游戏往有利于的方向进行, 最终赢得比赛. 

决策满足下面要求:
- 内容不要罗嗦, 不要超过50字数限制,少讲废话, 突出重点
- 判断场上信息真伪, 运用辩解,对抗,欺骗,伪装,坦白等等任意策略来做决策
- 决策分为两类:思考或行动
- 思考:逐步思考,判断信息真伪,分析游戏形势等等,参考 <reflections> 选择合适的输出
- 行动:参考<references>按照json字符串格式输出,必须包含action key, action必须在<commands>中选择


{input}
""".replace("{game_rule}", werewolf_rule_v1).replace("{commands}", werewolf_command_v1)

print(template_player_role)

In [ ]:
from shared.LangchainMini.LangchainMini import LangchainMini, LangchainMiniMemory, LangchainMiniPromptTemplate

# A和C是同学,确认说"ok"
# B和C是亲戚,确认说"ok"
# A和C会是什么关系,开始推理

# template = LangchainMiniPromptTemplate('''历史记录:{history}
# B和C是什么关系，开始推理"
# ''')

#template = LangchainMiniPromptTemplate(template_player_role)
#prompt = template.format(input="你是狼人，第一个夜晚如何行动?")

agent = LangchainMini(model_id="anthropic.claude-3-sonnet-20240229-v1:0", stream=True)
memory = LangchainMiniMemory(k=1, llm=agent)
llm = LangchainMini(model_id="anthropic.claude-3-sonnet-20240229-v1:0", stream=True, memory=memory)
# llm.System("Answer the question in Japanese.")

templates = ['{history} A和C是同学,确认说"ok"', '{history} B和C是亲戚,确认说"ok"', '{history} A和C会是什么关系,开始推理']
for template in templates:
    _template = LangchainMiniPromptTemplate(template)
    prompt = _template.format()
    llm.invoke(prompt)

In [ ]:
from shared.GameAssistant import GameAssistant
from shared.PeTemplates import *

#ga = GameAssistant(template_assistant_role, None)

#ga.DoAnswer("总结下内容")

# print(werewolf_command_v1)

# print(template_wolf_role)

# print(template_prophet_role)

# print(template_player_role)

# LoadPlayerPrompts()

# print(roles_dict)

def sort_players_by_role(players):
    # 定义角色优先级字典
    role_priorities = {
        "狼人": 1,
        "预言家": 2,
        "女巫": 3,
        "村民": 4
    }

    # 根据角色优先级对玩家列表进行排序
    sorted_players = sorted(players, key=lambda player: role_priorities.get(player["身份"], 5))

    return sorted_players

players = [
    {"名字": "张三", "身份": "狼人"},
    {"名字": "李四", "身份": "村民"},
    {"名字": "王五", "身份": "预言家"},
    {"名字": "赵六", "身份": "女巫"},
    {"名字": "钱七", "身份": "村民"}
]

sorted_players = sort_players_by_role(players)
print(sorted_players)

In [ ]:
from shared.GameServer import GameServer

GS = GameServer()
GS.Run("0.0.0.0", 8000)


In [ ]:
from shared.GamePlayer import GamePlayer
from shared.GameMaster import GameMaster
from shared.PeTemplates import *
LoadPlayerPrompts()
GM = GameMaster(1, False)
for player in roles_dict["players"]:
    _player = GamePlayer(player, GM)
    
    _player.DoPlanning(game_config_dict["player"]["action_plan_night"], 1)


In [2]:
from shared.GameMaster import GameMaster
from shared.PeTemplates import *

GM = GameMaster(1, 10, False)
GM.ResetGame()
GM.RunGame()
GM.EndGame()

04/12/2024 01:10:05 PM 	===== P1:存活,P2:存活,P3:存活,P4:存活,P5:淘汰,P6:存活,P7:存活,P8:存活 ResetGame =====
04/12/2024 01:10:05 PM P1 is 预言家
04/12/2024 01:10:05 PM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
04/12/2024 01:10:05 PM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
04/12/2024 01:10:05 PM P2 is 女巫
04/12/2024 01:10:05 PM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
04/12/2024 01:10:05 PM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
04/12/2024 01:10:05 PM P3 is 狼人
04/12/2024 01:10:05 PM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
04/12/2024 01:10:05 PM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
04/12/2024 01:10:05 PM P4 is 村民
04/12/2024 01:10:05 PM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
04/12/2024 01:10:05 PM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
04/12/2024 01:10:05 PM P5 is 村民
04/12/2024 01:10:05 PM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
04/12/2024 01:10:05 PM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
04/12/2024 01:1

Tought: 作为狼人阵营的组织者,我必须谨慎行事,在第一晚尽量不暴露身份。我应该选择一个看似无辜的村民作为目标,比如P8,从而误导好人阵营的怀疑方向。同时我需要和队友P6达成一致,以避免分裂投票。

Action: WolfVote
Action Input: agent=P3, target=P8
Final Answer: {"action": "WolfVote", "agent": "P3", "target": "P8"}

04/12/2024 01:10:15 PM 		******** DoAction NIGHT 1 P3 ********
04/12/2024 01:10:15 PM 
 [ROUND ACTION]={'time': 'NIGHT 1', 'player': 'P3', 'status': 1, 'role': '狼人', 'character': '组织者', 'response': ['{"action": "WolfVote", "agent": "P3", "target": "P8"}']}

04/12/2024 01:10:20 PM 		******** DoMemory NIGHT 1 P6 ********
04/12/2024 01:10:20 PM ['{"time": "NIGHT 1", "player": "P3", "status": 1, "response": {"action": "WolfVote", "agent": "P3", "target": "P8"}}']
04/12/2024 01:10:20 PM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
04/12/2024 01:10:20 PM HTTP Request: POST https://bedrock-runtime.us-east-1.amazonaws.com/model/anthropic.claude-3-sonnet-20240229-v1:0/invoke-with-response-stream "HTTP/1.1 200 OK"


夜晚第一夜,狼人P3投票杀死了村民P8。女巫可以选择是否使用解药或毒药来影响结果。预言家可以查验一名玩家的身份。

04/12/2024 01:10:27 PM 		******** DoAnswer NIGHT 1 P6 ********
04/12/2024 01:10:27 PM 	QUESTION: 目前场上玩家信息:P1:存活,P2:存活,P3:存活,P4:存活,P5:存活,P6:存活,P7:存活,P8:存活.记住，你支持的玩家是P6, 身份是狼人, 性格为过激型.接下来你必须帮助玩家进行这个游戏通过一步一步思考决策.本阵营玩家为:P3,P6.现在是第1天夜晚，该如何行动?
04/12/2024 01:10:27 PM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
04/12/2024 01:10:27 PM HTTP Request: POST https://bedrock-runtime.us-east-1.amazonaws.com/model/anthropic.claude-3-sonnet-20240229-v1:0/invoke-with-response-stream "HTTP/1.1 200 OK"


...

04/12/2024 01:10:33 PM 		******** DoAction NIGHT 1 P6 ********
04/12/2024 01:10:33 PM 
 [ROUND ACTION]={'time': 'NIGHT 1', 'player': 'P6', 'status': 1, 'role': '狼人', 'character': '过激型', 'response': ['{"agent": "P6", "target": "P8"}', '{"action": "WolfVote", "agent": "P6", "target": "P8"}']}

04/12/2024 01:10:38 PM 		******** DoMemory NIGHT 1 P1 ********
04/12/2024 01:10:43 PM 		******** DoAnswer NIGHT 1 P1 ********
04/12/2024 01:10:43 PM 	QUESTION: 目前场上玩家信息:P1:存活,P2:存活,P3:存活,P4:存活,P5:存活,P6:存活,P7:存活,P8:存活.记住，你支持的玩家是P1, 身份是预言家, 性格为独立思考.接下来你必须帮助玩家进行这个游戏通过一步一步思考决策.现在是第1天夜晚，该如何行动?
04/12/2024 01:10:43 PM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
04/12/2024 01:10:43 PM HTTP Request: POST https://bedrock-runtime.us-east-1.amazonaws.com/model/anthropic.claude-3-sonnet-20240229-v1:0/invoke-with-response-stream "HTTP/1.1 200 OK"


Tought: 作为预言家,我应该利用这个机会查验一名玩家的身份,以获取有利信息。我会选择一个行为较为可疑的玩家进行查验,比如在白天讨论时表现得较为支支吾吾、逻辑混乱或者过于激进的玩家。查验后我会根据结果进一步分析局势。

Action: ProphetCheck
Action Input: agent="P1", target="P4"  
Final Answer: {"action": "ProphetCheck", "agent": "P1", "target": "P4"}

04/12/2024 01:10:50 PM 		******** DoAction NIGHT 1 P1 ********
04/12/2024 01:10:50 PM [玩家P1于时间NIGHT 1, 执行动作为:{'action': 'ProphetCheck', 'agent': 'P1', 'target': 'P4'}] 结果 P4:村民
04/12/2024 01:10:50 PM 
 [ROUND ACTION]={'time': 'NIGHT 1', 'player': 'P1', 'status': 1, 'role': '预言家', 'character': '独立思考', 'response': ['{"action": "ProphetCheck", "agent": "P1", "target": "P4"}']}

04/12/2024 01:10:55 PM 		******** DoMemory NIGHT 1 P2 ********
04/12/2024 01:11:00 PM 		******** DoAnswer NIGHT 1 P2 ********
04/12/2024 01:11:00 PM 	QUESTION: 目前场上玩家信息:P1:存活,P2:存活,P3:存活,P4:存活,P5:存活,P6:存活,P7:存活,P8:存活.药水道具状态:毒药1,解药1记住，你支持的玩家是P2, 身份是女巫, 性格为腼腆型.接下来你必须帮助玩家进行这个游戏通过一步一步思考决策.现在是第1天夜晚，该如何行动?
04/12/2024 01:11:00 PM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
04/12/2024 01:11:00 PM HTTP Request: POST https://bedrock-runtime.us-east-1.amazonaws.com/model/anthropic.claude-3-sonnet-20240229-v1:0/invoke-with-response-stream "HTTP/1.1 200 OK"


Tought: 作为女巫,第一晚最明智的做法是不使用任何药水,观察形势。使用毒药或解药都可能引起不必要的怀疑和矛盾,影响游戏前景。保留两种药水以备不时之需是较为稳妥的策略。

Action: None
Action Input: None
Final Answer: None

04/12/2024 01:11:03 PM 		******** DoAction NIGHT 1 P2 ********
04/12/2024 01:11:03 PM 
 [ROUND ACTION]={'time': 'NIGHT 1', 'player': 'P2', 'status': 1, 'role': '女巫', 'character': '腼腆型', 'response': []}

04/12/2024 01:11:08 PM 	===== NIGHT 1 PostAction ======
04/12/2024 01:11:08 PM 	 [wolf_votes]: ['P8'], [player_vote_name]: Counter({'P8': 2})
04/12/2024 01:11:08 PM 
 [WOLF VOTE]={'time': 'NIGHT 1', 'player': 'P8', 'status': 0, 'role': '村民', 'character': '互动达人', 'response': '玩家P8于NIGHT 1被狼人投票而出局'}

04/12/2024 01:11:08 PM 游戏继续. 目前玩家存活情况:P1:存活,P2:存活,P3:存活,P4:存活,P5:存活,P6:存活,P7:存活,P8:淘汰
04/12/2024 01:11:08 PM 	===== input_tokens: 0 output_tokens 0 ======
04/12/2024 01:11:08 PM 	===== elapsed_time: 63.20775508880615 ======
04/12/2024 01:11:08 PM 游戏结束.
04/12/2024 01:11:08 PM ===== P1:存活,P2:存活,P3:存活,P4:存活,P5:存活,P6:存活,P7:存活,P8:淘汰 EndGame =====
04/12/2024 01:11:08 PM 	===== input_tokens: 0 output_tokens 0 ======
04/12/2024 01:11:08 PM 	===== elapsed_time: 63.2085280418396 ======
